In [1]:
!pip install -q -U openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from google.colab import userdata
api_key = userdata.get("OPENROUTER_API_KEY")
model = "mistralai/mistral-small-3.1-24b-instruct:free"
BASE_URL="https://openrouter.ai/api/v1"

In [5]:
from agents import Agent , Runner , AsyncOpenAI , OpenAIChatCompletionsModel , set_tracing_disabled
from agents.run import RunConfig

provider = AsyncOpenAI(
    api_key=api_key,
    base_url=BASE_URL,
)

model = OpenAIChatCompletionsModel(
    openai_client=provider,
    model=model,
)

config = RunConfig(
    model=model,
    tracing_disabled=True,
)

In [6]:
set_tracing_disabled(True)

In [11]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
async def main():

  agent = Agent(
      name="fact_generator",
      instructions="Generate A Facts Accoring to the user query.",
      model=model,
  )

  result = Runner.run_streamed(agent , "We Love Cricket Tell me 5 Fun facts about Cricket" , run_config=config)
  async for event in result.stream_events():
      if event.type == "raw_response_event" and isinstance(event.data , ResponseTextDeltaEvent):
        print(event.data.delta , end="" , flush=True)

asyncio.run(main())



Here are five fun facts about cricket:

1. **Oldest Organized Sport**: Cricket is believed to be the oldest organized sport in the world, with its origins dating back to the 16th century. The first written evidence of cricket can be found in a 1598 court case in England, where a plot of land was noted as being used for cricket by locals for "time out of mind."

2. **Longest Game Ever**: The longest cricket match ever played lasted for 14 days, with the actual playing time being about 12 days. It was a two-innings-per-side game between New Zealand and South Africa at Wellington in 1991-92.

3. **Smallest Ground**: Lord's Cricket Ground in London, often referred to as the "Home of Cricket," is the smallest international cricket ground by boundary size, with the shortest boundary being just 65 yards (59 meters).

4. **Most Wickets in Tests**: As of my knowledge cutoff in 2023, Muttiah Muralitharan of Sri Lanka holds the record for the most wickets in Test cricket, with 800 wickets in 133 

# streaming with tool usage

In [15]:
from agents import Agent , Runner , function_tool , ItemHelpers
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
import random

@function_tool
def how_many_jokes():
       return random.randint(1 , 10)

async def main():

  agent = Agent(
      name="fact_generator",
      instructions="Generate A Facts According to the user Query.",
      tools=[how_many_jokes],
      model=model,
  )

  result = Runner.run_streamed(agent , "We Love Cricket Tell me facts about Cricket" , run_config=config)
  async for event in result.stream_events():
    if event.type == "raw_response_event":
      continue
    elif event.type == "agent_updated_stream_event":
      print(f"Agent Updated :  {event.new_agent.name}")
      continue
    elif event.type == "run_item_stream_event":
      if event.item.type == "tool_call_item":
        print(" - - - tool was Called")
      elif event.item.type == "tool_call_output_item":
        print(f"-- tool Output : {event.item.output}")
      elif event.item.type == "message_output_item":
        print(f"- - Message output :\n {ItemHelpers.text_message_output(event.item)}")
      else:
        pass



asyncio.run(main())




Agent Updated :  fact_generator
- - Message output :
 Here are some interesting facts about cricket:

1. Origin and History:
   Cricket originated in England, with the earliest known references dating back to the 16th century. It was typically
   played as a children's game, but it evolved into a more structured sport over time.

2. Rules and Format:
   Cricket is played between two teams, each consisting of 11 players. The game is played on a field with a pitch in the
   center. The objective is to score runs while batting and to dismiss the opposing team's batsmen while bowling.

3. Test Cricket:
   Test cricket is the longest format of the game, lasting up to five days. The first Test match was played between Australia and England in 1877.
   The rivalry between these two teams is known as The Ashes, a prestigious competition played every two years.

4. One Day Internationals (ODIs):
   ODIs are 50-over matches per side, typically completed in a day. The first ODI was played in 1971